# Lectura del dataset

##### Paso 1: Lectura del dataset "b-mc2/sql-create-context"

In [416]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")

import datasets

dataset = datasets.load_dataset("b-mc2/sql-create-context")
df = dataset["train"].to_pandas()

df.head()

df_copy = df.copy()

def filter_by_join_count(df, difficulty, number_of_samples):
    """Filtrado de ejemplos del dataset por cantidad de JOINS en el ejemplo"""
    if difficulty == "lvl1":
        join_mask = df["answer"].str.count("JOIN") == 0
    elif difficulty == "lvl2":
        join_mask = df["answer"].str.count("JOIN") == 1
    elif difficulty == "lvl3":
        join_mask = df["answer"].str.count("JOIN") == 2
    elif difficulty == "lvl4":
        join_mask = df["answer"].str.count("JOIN") >= 4
    else:
        raise ValueError("Invalid difficulty level")

    join_df = df[join_mask]
    return join_df.sample(n=number_of_samples, random_state=6)

##### Paso 2: Filtrado del dataset por cantidad de joins en los ejemplos

In [417]:
lvl1_df = filter_by_join_count(df_copy, "lvl1", 30)
lvl2_df = filter_by_join_count(df_copy, "lvl2", 30)
lvl3_df = filter_by_join_count(df_copy, "lvl3", 30)
lvl4_df = filter_by_join_count(df_copy, "lvl4", 30)

# Buscamos las respuestas (posteriormente usadas para generar los joins con chatgpt)
responses_lvl1 = lvl1_df["answer"].tolist()
responses_lvl2 = lvl2_df["answer"].tolist()
responses_lvl3 = lvl3_df["answer"].tolist()
responses_lvl4 = lvl4_df["answer"].tolist()
# Buscamos los inputs (posteriormente usadas para generar los comentarios con chatgpt)
context_lvl1 = lvl1_df["context"].tolist()
context_lvl2 = lvl2_df["context"].tolist()
context_lvl3 = lvl3_df["context"].tolist()
context_lvl4 = lvl4_df["context"].tolist()

# Generación de relaciones y comentarios con chatgpt
##### Como no tenemos data con comentarios o relaciones entre tablas, usamos chatgpt para poder generarlos y agregarlos al set de datos

#### Paso 1: Generamos el prompt para chatgpt
##### Un prompt para relaciones y otro para comentarios

In [427]:
from train.mistral7bbase.functions.data_functions import get_gpt_relationships_prompt
import pyperclip

p = get_gpt_relationships_prompt(responses_lvl4)
pyperclip.copy(p)
# print(p)

#### Resultados de relaciones entre tablas

In [459]:
#####################
gpt_relations_lvl1 = []
gpt_relations_lvl2 = []
gpt_relations_lvl3 = [
'-- program.program_id can be joined with broadcast.program_id, -- program.program_id can be joined with broadcast.program_id',
'-- routes.dst_apid can be joined with airports.apid, -- routes.alid can be joined with airlines.alid',
'-- staff.staff_id can be joined with Staff_Department_Assignments.staff_id, -- staff.staff_id can be joined with Staff_Department_Assignments.staff_id',
'-- game.stadium_id can be joined with stadium.id, -- game.id can be joined with injury_accident.game_id',
'-- customers.customer_id can be joined with customer_addresses.customer_id, -- customer_addresses.address_id can be joined with addresses.address_id',
'-- Third_Party_Companies.company_id can be joined with Maintenance_Engineers.company_id, -- Third_Party_Companies.company_id can be joined with Maintenance_Contracts.maintenance_contract_company_id',
'-- concert.stadium_id can be joined with stadium.stadium_id, -- concert.stadium_id can be joined with stadium.stadium_id',
'-- player_award.year can be joined with player_award.year, -- player_award.year can be joined with player_award.year',
'-- student.stuid can be joined with lives_in.stuid, -- dorm.dormid can be joined with lives_in.dormid',
'-- LOTS.lot_id can be joined with TRANSACTIONS_LOTS.transaction_id, -- TRANSACTIONS_LOTS.transaction_id can be joined with TRANSACTIONS.transaction_id',
'-- club.clubid can be joined with member_of_club.clubid, -- member_of_club.stuid can be joined with student.stuid',
'-- customer_address_history.customer_id can be joined with customers.customer_id, -- customer_address_history.address_id can be joined with addresses.address_id',
'-- participates_in.actid can be joined with activity.actid, -- participates_in.actid can be joined with activity.actid',
'-- club.clubid can be joined with member_of_club.clubid, -- member_of_club.stuid can be joined with student.stuid',
'-- club.clubid can be joined with member_of_club.clubid, -- member_of_club.stuid can be joined with student.stuid',
'-- MODEL_LIST.Maker can be joined with CAR_MAKERS.Id, -- CAR_MAKERS.Country can be joined with COUNTRIES.CountryId',
'-- Faculty.facID can be joined with Faculty_participates_in.facID, -- Faculty_participates_in.actid can be joined with Activity.actid',
'-- club.clubid can be joined with member_of_club.clubid, -- member_of_club.stuid can be joined with student.stuid',
'-- mailshot_customers.customer_id can be joined with customers.customer_id, -- mailshot_customers.customer_id can be joined with customers.customer_id',
'-- procedures.cost can be joined with physician.employeeid, -- procedures.code can be joined with trained_in.treatment',
'-- CLASS.crs_code can be joined with course.crs_code, -- course.dept_code can be joined with department.dept_code',
'-- genres.id can be joined with tracks.genre_id, -- media_types.id can be joined with tracks.media_type_id',
'-- Performance.bandmate can be joined with Band.id, -- Performance.SongId can be joined with Songs.SongId',
'-- products.Manufacturer can be joined with manufacturers.code, -- products.Manufacturer can be joined with manufacturers.code',
'-- services.service_id can be joined with EVENTS.service_id, -- services.service_id can be joined with EVENTS.service_id',
'-- customers.customer_id can be joined with customer_addresses.customer_id, -- customer_addresses.address_id can be joined with addresses.address_id',
'-- accounts.custid can be joined with checking.custid, -- accounts.custid can be joined with savings.custid',
'-- undergoes.patient can be joined with patient.SSN, -- undergoes.Stay can be joined with stay.StayID',
'-- department.dept_name can be joined with student.dept_name, -- department.dept_name can be joined with instructor.dept_name',
'-- Allergy_type.Allergy can be joined with Has_allergy.Allergy, -- Has_allergy.StuID can be joined with Student.StuID',
]
gpt_relations_lvl4 = [
'-- Tourist_Attractions.Tourist_Attraction_ID can be joined with VISITS.Tourist_Attraction_ID, -- VISITORS.Tourist_ID can be joined with VISITS.Tourist_ID',
'-- web_client_accelerator.id can be joined with accelerator_compatible_browser.accelerator_id, -- browser.id can be joined with accelerator_compatible_browser.browser_id',
'-- tracks.id can be joined with playlist_tracks.track_id, -- playlists.id can be joined with playlist_tracks.playlist_id',
'-- branch.branch_id can be joined with membership_register_branch.branch_id, -- member.member_id can be joined with membership_register_branch.member_id',
'-- department.department_id can be joined with management.department_id, -- head.head_id can be joined with management.head_id',
'-- store.store_id can be joined with store_district.store_id, -- district.district_id can be joined with store_district.district_id',
'-- problems.product_id can be joined with product.product_id, -- staff.staff_id can be joined with problems.reported_by_staff_id',
'-- student.stuid can be joined with has_pet.stuid, -- pets.petid can be joined with has_pet.petid',
'-- Grants.grant_id can be joined with Documents.grant_id, -- Document_Types.document_type_code can be joined with Documents.document_type_code',
'-- Tourist_Attractions.tourist_attraction_id can be joined with Tourist_Attraction_Features.tourist_attraction_id, -- Features.Feature_ID can be joined with Tourist_Attraction_Features.Feature_ID',
'-- Faculty.facID can be joined with Faculty_participates_in.facID, -- activity.actid can be joined with Faculty_participates_in.actid',
'-- game.stadium_id can be joined with stadium.id, -- game.id can be joined with injury_accident.game_id',
'-- visitor.id can be joined with visit.visitor_id, -- museum.Museum_ID can be joined with visit.Museum_ID',
'-- contestants.contestant_number can be joined with votes.contestant_number, -- area_code_state.state can be joined with votes.state',
'-- Tourist_Attractions.Tourist_Attraction_ID can be joined with VISITS.Tourist_Attraction_ID, -- VISITORS.Tourist_ID can be joined with VISITS.Tourist_ID',
'-- customers.customer_id can be joined with first_notification_of_loss.customer_id, -- services.service_id can be joined with first_notification_of_loss.service_id',
'-- aircraft.Aircraft_ID can be joined with airport_aircraft.Aircraft_ID, -- airport.Airport_ID can be joined with airport_aircraft.Airport_ID',
'-- dorm.dormid can be joined with has_amenity.dormid, -- dorm_amenity.amenid can be joined with has_amenity.amenid',
'-- tracks.id can be joined with playlist_tracks.track_id, -- playlists.id can be joined with playlist_tracks.playlist_id',
'-- races.raceid can be joined with results.raceid, -- drivers.driverid can be joined with results.driverid',
'-- dorm.dormid can be joined with has_amenity.dormid, -- dorm_amenity.amenid can be joined with has_amenity.amenid',
'-- Album.AlbumId can be joined with Track.AlbumId, -- Genre.GenreID can be joined with Track.GenreID',
'-- salary.player_id can be joined with player.player_id, -- team.team_id_br can be joined with salary.team_id',
'-- CLASS.prof_num can be joined with employee.emp_num, -- CLASS.crs_code can be joined with course.crs_code, -- professor.emp_num can be joined with employee.emp_num, -- department.dept_code can be joined with professor.dept_code',
'-- customers.customer_id can be joined with customer_orders.customer_id, -- products.product_id can be joined with order_items.product_id, -- customer_orders.order_id can be joined with order_items.order_id',
'-- student.stuid can be joined with has_pet.stuid, -- pets.petid can be joined with has_pet.petid',
'-- train_station.station_id can be joined with station.station_id, -- train.train_id can be joined with train_station.train_id',
'-- furniture.Furniture_ID can be joined with furniture_manufacte.Furniture_ID, -- manufacturer.manufacturer_id can be joined with furniture_manufacte.manufacturer_id',
'-- student.stu_num can be joined with enroll.stu_num, -- CLASS.class_code can be joined with enroll.class_code, -- course.crs_code can be joined with CLASS.crs_code, -- department.dept_code can be joined with course.dept_code',
'-- Employee.eid can be joined with Certificate.eid, -- Aircraft.aid can be joined with Certificate.aid',
]



print(len(gpt_relations_lvl1), len(gpt_relations_lvl2), len(gpt_relations_lvl3), len(gpt_relations_lvl4))

0 0 30 30


#### Resultados de comentarios en las tablas

In [456]:
from train.mistral7bbase.functions.data_functions import get_gpt_comments_prompt
import pyperclip

# p = get_gpt_comments_prompt(context_lvl1)

p = get_gpt_comments_prompt(context_lvl4[:1])
p = get_gpt_comments_prompt(context_lvl4[1:3])
p = get_gpt_comments_prompt(context_lvl4[3:6])
p = get_gpt_comments_prompt(context_lvl4[6:10])
p = get_gpt_comments_prompt(context_lvl4[10:15])
p = get_gpt_comments_prompt(context_lvl4[15:20])
p = get_gpt_comments_prompt(context_lvl4[20:25])
p = get_gpt_comments_prompt(context_lvl4[25:30])

pyperclip.copy(p)
# print(p)

In [430]:
gpt_context_lvl1 = []

In [431]:
gpt_context_lvl2 = []
gpt_context_lvl2 += []
gpt_context_lvl2 += []
gpt_context_lvl2 += []
gpt_context_lvl2 += []
gpt_context_lvl2 += []
gpt_context_lvl2 += []



In [460]:
gpt_context_lvl3 = [
    '''
    CREATE TABLE IF NOT EXISTS broadcast (
        program_id VARCHAR, -- Identifier of the program being broadcasted
        Time_of_day VARCHAR -- Time of day the program is scheduled to be broadcasted
    );
    CREATE TABLE IF NOT EXISTS program (
        owner VARCHAR, -- Owner of the program
        program_id VARCHAR -- Identifier of the program
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS routes (
        dst_apid VARCHAR, -- Destination airport ID
        alid VARCHAR -- Airline ID
    );
    CREATE TABLE IF NOT EXISTS airports (
        apid VARCHAR, -- Airport ID
        country VARCHAR -- Country where the airport is located
    );
    CREATE TABLE IF NOT EXISTS airlines (
        alid VARCHAR, -- Airline ID
        name VARCHAR -- Name of the airline
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS staff (
        staff_name VARCHAR, -- Name of the staff member
        staff_gender VARCHAR, -- Gender of the staff member
        staff_id VARCHAR -- Unique identifier for the staff member
    );
    CREATE TABLE IF NOT EXISTS Staff_Department_Assignments (
        staff_id VARCHAR, -- Unique identifier for the staff member
        job_title_code VARCHAR -- Job title code for the staff member's department assignment
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS stadium (
        name VARCHAR, -- Name of the stadium
        average_attendance VARCHAR, -- Average attendance at the stadium
        total_attendance VARCHAR, -- Total attendance at the stadium
        id VARCHAR -- Unique identifier for the stadium
    );
    CREATE TABLE IF NOT EXISTS stadium (
        name VARCHAR, -- Name of the stadium
        average_attendance VARCHAR, -- Average attendance at the stadium
        total_attendance VARCHAR -- Total attendance at the stadium
    );
    CREATE TABLE IF NOT EXISTS game (
        stadium_id VARCHAR, -- Identifier of the stadium where the game is held
        id VARCHAR -- Unique identifier for the game
    );
    CREATE TABLE IF NOT EXISTS injury_accident (
        game_id VARCHAR -- Identifier of the game where the injury or accident occurred
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS customer_addresses (
        customer_id VARCHAR, -- Identifier of the customer
        address_id VARCHAR -- Identifier of the address
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR, -- Name of the customer
        customer_phone VARCHAR, -- Phone number of the customer
        customer_id VARCHAR -- Unique identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS addresses (
        address_id VARCHAR, -- Unique identifier for the address
        state_province_county VARCHAR -- State, province, or county of the address
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Maintenance_Contracts (
        maintenance_contract_company_id VARCHAR -- Identifier of the maintenance contract company
    );
    CREATE TABLE IF NOT EXISTS Maintenance_Engineers (
        company_id VARCHAR -- Identifier of the maintenance engineer's company
    );
    CREATE TABLE IF NOT EXISTS Third_Party_Companies (
        company_id VARCHAR, -- Unique identifier for the third-party company
        company_name VARCHAR -- Name of the third-party company
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS stadium (
        name VARCHAR, -- Name of the stadium
        location VARCHAR, -- Location of the stadium
        stadium_id VARCHAR -- Unique identifier for the stadium
    );
    CREATE TABLE IF NOT EXISTS concert (
        stadium_id VARCHAR, -- Identifier of the stadium where the concert takes place
        Year VARCHAR -- Year of the concert
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS player (
        name_first VARCHAR, -- First name of the player
        name_last VARCHAR -- Last name of the player
    );
    CREATE TABLE IF NOT EXISTS player_award (
        year VARCHAR -- Year the award was received
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS dorm (
        dorm_name VARCHAR, -- Name of the dormitory
        dormid VARCHAR -- Unique identifier for the dormitory
    );
    CREATE TABLE IF NOT EXISTS student (
        age INTEGER, -- Age of the student
        stuid VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS lives_in (
        stuid VARCHAR, -- Unique identifier for the student
        dormid VARCHAR -- Unique identifier for the dormitory where the student lives
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS LOTS (
        lot_details VARCHAR, -- Details of the lot
        lot_id VARCHAR -- Unique identifier for the lot
    );
    CREATE TABLE IF NOT EXISTS TRANSACTIONS_LOTS (
        transaction_id VARCHAR -- Unique identifier for the transaction related to the lot
    );
    CREATE TABLE IF NOT EXISTS TRANSACTIONS (
        transaction_id VARCHAR, -- Unique identifier for the transaction
        share_count INTEGER -- Number of shares involved in the transaction
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS student (
        stuid VARCHAR, -- Unique identifier for the student
        sex VARCHAR -- Gender of the student
    );
    CREATE TABLE IF NOT EXISTS member_of_club (
        clubid VARCHAR, -- Unique identifier for the club
        stuid VARCHAR -- Unique identifier for the student who is a member
    );
    CREATE TABLE IF NOT EXISTS club (
        clubname VARCHAR, -- Name of the club
        clubid VARCHAR -- Unique identifier for the club
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS addresses (
        city VARCHAR, -- Name of the city
        address_id VARCHAR -- Unique identifier for the address
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR, -- Name of the customer
        customer_id VARCHAR -- Unique identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS customer_address_history (
        date_from VARCHAR, -- Start date of customer's residence at the address
        date_to VARCHAR, -- End date of customer's residence at the address
        customer_id VARCHAR, -- Unique identifier for the customer
        address_id VARCHAR -- Unique identifier for the address
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS activity (
        actid VARCHAR, -- Unique identifier for the activity
        activity_name VARCHAR -- Name of the activity
    );
    CREATE TABLE IF NOT EXISTS participates_in (
        stuid VARCHAR -- Unique identifier for the student participating in the activity
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS club (
        clubid VARCHAR, -- Unique identifier for the club
        clubname VARCHAR -- Name of the club
    );
    CREATE TABLE IF NOT EXISTS member_of_club (
        clubid VARCHAR, -- Unique identifier for the club
        stuid VARCHAR -- Unique identifier for the student who is a member
    );
    CREATE TABLE IF NOT EXISTS student (
        stuid VARCHAR -- Unique identifier for the student
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS club (
        clubid VARCHAR, -- Unique identifier for the club
        clubname VARCHAR -- Name of the club
    );
    CREATE TABLE IF NOT EXISTS student (
        age INTEGER, -- Age of the student
        stuid VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS member_of_club (
        clubid VARCHAR, -- Unique identifier for the club
        stuid VARCHAR -- Unique identifier for the student who is a member
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS CAR_MAKERS (
        Id VARCHAR, -- Unique identifier for the car maker
        Country VARCHAR -- Country of the car maker
    );
    CREATE TABLE IF NOT EXISTS COUNTRIES (
        CountryId VARCHAR, -- Unique identifier for the country
        CountryName VARCHAR -- Name of the country
    );
    CREATE TABLE IF NOT EXISTS MODEL_LIST (
        Maker VARCHAR -- Name of the car maker
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Activity (
        activity_name VARCHAR, -- Name of the activity
        actid VARCHAR -- Unique identifier for the activity
    );
    CREATE TABLE IF NOT EXISTS Faculty (
        facID VARCHAR, -- Unique identifier for the faculty
        fname VARCHAR, -- First name of the faculty
        lname VARCHAR -- Last name of the faculty
    );
    CREATE TABLE IF NOT EXISTS Faculty_participates_in (
        facID VARCHAR, -- Unique identifier for the faculty
        actid VARCHAR -- Unique identifier for the activity the faculty participates in
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS club (
        clubid VARCHAR, -- Unique identifier for the club
        clubname VARCHAR -- Name of the club
    );
    CREATE TABLE IF NOT EXISTS student (
        stuid VARCHAR, -- Unique identifier for the student
        age VARCHAR -- Age of the student
    );
    CREATE TABLE IF NOT EXISTS member_of_club (
        clubid VARCHAR, -- Unique identifier for the club
        stuid VARCHAR -- Unique identifier for the student who is a member
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR, -- Name of the customer
        payment_method VARCHAR, -- Payment method used by the customer
        customer_id VARCHAR -- Unique identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS mailshot_customers (
        customer_id VARCHAR, -- Unique identifier for the customer
        outcome_code VARCHAR -- Outcome code of the mailshot
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS trained_in (
        physician VARCHAR, -- Name of the physician
        treatment VARCHAR -- Treatment trained in
    );
    CREATE TABLE IF NOT EXISTS procedures (
        name VARCHAR, -- Name of the procedure
        cost INTEGER -- Cost of the procedure
    );
    CREATE TABLE IF NOT EXISTS physician (
        employeeid VARCHAR, -- Unique identifier for the physician
        name VARCHAR -- Name of the physician
    );
    CREATE TABLE IF NOT EXISTS procedures (
        name VARCHAR, -- Name of the procedure
        code VARCHAR -- Code of the procedure
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS department (
        school_code VARCHAR, -- Code of the school
        dept_code VARCHAR -- Code of the department
    );
    CREATE TABLE IF NOT EXISTS CLASS (
        crs_code VARCHAR -- Course code
    );
    CREATE TABLE IF NOT EXISTS course (
        crs_code VARCHAR, -- Course code
        dept_code VARCHAR -- Code of the department offering the course
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS genres (
        id VARCHAR, -- Unique identifier for the genre
        name VARCHAR -- Name of the genre
    );
    CREATE TABLE IF NOT EXISTS tracks (
        name VARCHAR, -- Name of the track
        genre_id VARCHAR, -- Identifier of the genre of the track
        media_type_id VARCHAR -- Identifier of the media type of the track
    );
    CREATE TABLE IF NOT EXISTS media_types (
        id VARCHAR, -- Unique identifier for the media type
        name VARCHAR -- Name of the media type
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Songs (
        Title VARCHAR, -- Title of the song
        SongId VARCHAR -- Unique identifier for the song
    );
    CREATE TABLE IF NOT EXISTS Performance (
        bandmate VARCHAR, -- Name of the bandmate
        SongId VARCHAR -- Unique identifier for the song
    );
    CREATE TABLE IF NOT EXISTS Band (
        id VARCHAR, -- Unique identifier for the band
        Lastname VARCHAR -- Last name of the band
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS manufacturers (
        code VARCHAR, -- Manufacturer code
        name VARCHAR -- Name of the manufacturer
    );
    CREATE TABLE IF NOT EXISTS products (
        name VARCHAR, -- Name of the product
        Manufacturer VARCHAR -- Manufacturer of the product
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS EVENTS (
        service_id VARCHAR, -- Identifier for the service associated with the event
        event_details VARCHAR -- Details of the event
    );
    CREATE TABLE IF NOT EXISTS services (
        service_type_code VARCHAR, -- Code for the service type
        service_id VARCHAR -- Unique identifier for the service
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR, -- Unique identifier for the customer
        customer_name VARCHAR -- Name of the customer
    );
    CREATE TABLE IF NOT EXISTS customer_addresses (
        customer_id VARCHAR, -- Unique identifier for the customer
        address_id VARCHAR -- Unique identifier for the address
    );
    CREATE TABLE IF NOT EXISTS addresses (
        address_content VARCHAR, -- Content of the address
        address_id VARCHAR -- Unique identifier for the address
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS checking (
        balance INTEGER, -- Balance in the checking account
        custid VARCHAR -- Identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS savings (
        balance INTEGER, -- Balance in the savings account
        custid VARCHAR -- Identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS accounts (
        name VARCHAR, -- Name of the account
        custid VARCHAR -- Identifier for the customer
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS undergoes (
        patient VARCHAR, -- Name of the patient
        Stay VARCHAR -- Identifier for the stay
    );
    CREATE TABLE IF NOT EXISTS stay (
        StayID VARCHAR, -- Unique identifier for the stay
        room VARCHAR -- Room for the stay
    );
    CREATE TABLE IF NOT EXISTS patient (
        name VARCHAR, -- Name of the patient
        SSN VARCHAR -- Social Security Number of the patient
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS department (
        dept_name VARCHAR -- Name of the department
    );
    CREATE TABLE IF NOT EXISTS student (
        id VARCHAR, -- Unique identifier for the student
        dept_name VARCHAR -- Name of the department the student belongs to
    );
    CREATE TABLE IF NOT EXISTS instructor (
        dept_name VARCHAR, -- Name of the department
        id VARCHAR -- Unique identifier for the instructor
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Has_allergy (
        Allergy VARCHAR, -- Name of the allergy
        StuID VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS Student (
        StuID VARCHAR, -- Unique identifier for the student
        Fname VARCHAR -- First name of the student
    );
    CREATE TABLE IF NOT EXISTS Allergy_type (
        Allergy VARCHAR, -- Name of the allergy
        AllergyType VARCHAR -- Type of the allergy
    );
    '''
]

In [461]:
gpt_context_lvl4 = [
    '''
    CREATE TABLE IF NOT EXISTS VISITS (
        Tourist_Attraction_ID VARCHAR, -- Identifier for the visited tourist attraction
        Tourist_ID VARCHAR -- Identifier for the tourist visiting the attraction
    );
    CREATE TABLE IF NOT EXISTS VISITORS (
        Tourist_Details VARCHAR, -- Details of the tourist
        Tourist_ID VARCHAR -- Unique identifier for the tourist
    );
    CREATE TABLE IF NOT EXISTS Tourist_Attractions (
        Name VARCHAR, -- Name of the tourist attraction
        Tourist_Attraction_ID VARCHAR -- Unique identifier for the tourist attraction
    );
    '''
]

gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS accelerator_compatible_browser (
        accelerator_id VARCHAR, -- Identifier for the accelerator
        browser_id VARCHAR -- Identifier for the browser compatible with the accelerator
    );
    CREATE TABLE IF NOT EXISTS web_client_accelerator (
        id VARCHAR, -- Unique identifier for the web client accelerator
        name VARCHAR -- Name of the web client accelerator
    );
    CREATE TABLE IF NOT EXISTS browser (
        name VARCHAR, -- Name of the browser
        id VARCHAR -- Unique identifier for the browser
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS playlists (
        id VARCHAR, -- Unique identifier for the playlist
        name VARCHAR -- Name of the playlist
    );
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        track_id VARCHAR, -- Identifier for the track in the playlist
        playlist_id VARCHAR -- Identifier for the playlist containing the track
    );
    CREATE TABLE IF NOT EXISTS tracks (
        name VARCHAR, -- Name of the track
        id VARCHAR -- Unique identifier for the track
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS member (
        member_id VARCHAR, -- Unique identifier for the member
        Hometown VARCHAR -- Hometown of the member
    );
    CREATE TABLE IF NOT EXISTS branch (
        name VARCHAR, -- Name of the branch
        branch_id VARCHAR -- Unique identifier for the branch
    );
    CREATE TABLE IF NOT EXISTS membership_register_branch (
        branch_id VARCHAR, -- Unique identifier for the branch
        member_id VARCHAR -- Unique identifier for the member in the branch
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS management (
        department_id VARCHAR, -- Unique identifier for the department managed
        head_id VARCHAR -- Unique identifier for the head of the department
    );
    CREATE TABLE IF NOT EXISTS head (
        born_state VARCHAR, -- State where the head was born
        head_id VARCHAR -- Unique identifier for the head
    );
    CREATE TABLE IF NOT EXISTS department (
        department_id VARCHAR, -- Unique identifier for the department
        name VARCHAR -- Name of the department
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS district (
        District_name VARCHAR, -- Name of the district
        district_id VARCHAR -- Unique identifier for the district
    );
    CREATE TABLE IF NOT EXISTS store (
        store_id VARCHAR, -- Unique identifier for the store
        Type VARCHAR -- Type of the store
    );
    CREATE TABLE IF NOT EXISTS store_district (
        store_id VARCHAR, -- Unique identifier for the store
        district_id VARCHAR -- Unique identifier for the district
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS product (
        product_name VARCHAR, -- Name of the product
        product_id VARCHAR -- Unique identifier for the product
    );
    CREATE TABLE IF NOT EXISTS staff (
        staff_id VARCHAR, -- Unique identifier for the staff
        staff_first_name VARCHAR, -- First name of the staff
        staff_last_name VARCHAR -- Last name of the staff
    );
    CREATE TABLE IF NOT EXISTS problems (
        product_id VARCHAR, -- Unique identifier for the product
        reported_by_staff_id VARCHAR -- Unique identifier for the staff reporting the problem
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS has_pet (
        stuid VARCHAR, -- Unique identifier for the student
        petid VARCHAR -- Unique identifier for the pet
    );
    CREATE TABLE IF NOT EXISTS student (
        Fname VARCHAR, -- First name of the student
        stuid VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS pets (
        petid VARCHAR, -- Unique identifier for the pet
        pettype VARCHAR -- Type of the pet
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Grants (
        grant_start_date VARCHAR, -- Start date of the grant
        grant_id VARCHAR -- Unique identifier for the grant
    );
    CREATE TABLE IF NOT EXISTS Document_Types (
        document_type_code VARCHAR, -- Code for the document type
        document_description VARCHAR -- Description of the document type
    );
    CREATE TABLE IF NOT EXISTS Documents (
        grant_id VARCHAR, -- Unique identifier for the grant
        document_type_code VARCHAR -- Code for the document type
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Tourist_Attraction_Features (
        tourist_attraction_id VARCHAR, -- Unique identifier for the tourist attraction
        Feature_ID VARCHAR -- Unique identifier for the feature of the tourist attraction
    );
    CREATE TABLE IF NOT EXISTS Features (
        Feature_ID VARCHAR, -- Unique identifier for the feature
        feature_Details VARCHAR -- Details of the feature
    );
    CREATE TABLE IF NOT EXISTS Tourist_Attractions (
        Name VARCHAR, -- Name of the tourist attraction
        tourist_attraction_id VARCHAR -- Unique identifier for the tourist attraction
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS activity (
        activity_name VARCHAR -- Name of the activity
    );
    CREATE TABLE IF NOT EXISTS Faculty_participates_in (
        facID VARCHAR, -- Unique identifier for the faculty
        actid VARCHAR -- Unique identifier for the activity the faculty participates in
    );
    CREATE TABLE IF NOT EXISTS Faculty (
        lname VARCHAR, -- Last name of the faculty
        facID VARCHAR -- Unique identifier for the faculty
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS stadium (
        name VARCHAR, -- Name of the stadium
        id VARCHAR -- Unique identifier for the stadium
    );
    CREATE TABLE IF NOT EXISTS game (
        stadium_id VARCHAR, -- Unique identifier for the stadium
        id VARCHAR -- Unique identifier for the game
    );
    CREATE TABLE IF NOT EXISTS injury_accident (
        game_id VARCHAR, -- Unique identifier for the game
        player VARCHAR -- Name of the player involved in the injury/accident
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS visitor (
        name VARCHAR, -- Name of the visitor
        id VARCHAR -- Unique identifier for the visitor
    );
    CREATE TABLE IF NOT EXISTS museum (
        Museum_ID VARCHAR, -- Unique identifier for the museum
        open_year INTEGER -- Year when the museum opened
    );
    CREATE TABLE IF NOT EXISTS visit (
        visitor_id VARCHAR, -- Unique identifier for the visitor
        Museum_ID VARCHAR -- Unique identifier for the museum visited
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS votes (
        contestant_number VARCHAR, -- Number of the contestant
        state VARCHAR -- State where the vote is cast
    );
    CREATE TABLE IF NOT EXISTS contestants (
        contestant_number VARCHAR, -- Number of the contestant
        contestant_name VARCHAR -- Name of the contestant
    );
    CREATE TABLE IF NOT EXISTS area_code_state (
        area_code VARCHAR, -- Area code
        state VARCHAR -- State associated with the area code
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS VISITS (
        Tourist_Attraction_ID VARCHAR, -- Identifier for the visited tourist attraction
        Tourist_ID VARCHAR -- Identifier for the tourist visiting the attraction
    );
    CREATE TABLE IF NOT EXISTS VISITORS (
        Tourist_Details VARCHAR, -- Details of the tourist
        Tourist_ID VARCHAR -- Unique identifier for the tourist
    );
    CREATE TABLE IF NOT EXISTS Tourist_Attractions (
        Name VARCHAR, -- Name of the tourist attraction
        Tourist_Attraction_ID VARCHAR -- Unique identifier for the tourist attraction
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS first_notification_of_loss (
        customer_id VARCHAR, -- Unique identifier for the customer reporting the loss
        service_id VARCHAR -- Unique identifier for the service related to the loss
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR, -- Name of the customer
        customer_id VARCHAR -- Unique identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS services (
        service_id VARCHAR, -- Unique identifier for the service
        service_name VARCHAR -- Name of the service
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS airport (
        Airport_ID VARCHAR, -- Unique identifier for the airport
        Airport_Name VARCHAR -- Name of the airport
    );
    CREATE TABLE IF NOT EXISTS aircraft (
        Aircraft VARCHAR, -- Type of aircraft
        Aircraft_ID VARCHAR -- Unique identifier for the aircraft
    );
    CREATE TABLE IF NOT EXISTS airport_aircraft (
        Aircraft_ID VARCHAR, -- Unique identifier for the aircraft
        Airport_ID VARCHAR -- Unique identifier for the airport
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS dorm (
        dorm_name VARCHAR, -- Name of the dorm
        dormid VARCHAR -- Unique identifier for the dorm
    );
    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid VARCHAR, -- Unique identifier for the dorm
        amenid VARCHAR -- Unique identifier for the amenity available in the dorm
    );
    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid VARCHAR, -- Unique identifier for the amenity
        amenity_name VARCHAR -- Name of the amenity available in the dorm
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS playlists (
        id VARCHAR, -- Unique identifier for the playlist
        name VARCHAR -- Name of the playlist
    );
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        track_id VARCHAR, -- Unique identifier for the track in the playlist
        playlist_id VARCHAR -- Unique identifier for the playlist containing the track
    );
    CREATE TABLE IF NOT EXISTS tracks (
        name VARCHAR, -- Name of the track
        id VARCHAR -- Unique identifier for the track
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS races (
        raceid VARCHAR, -- Unique identifier for the race
        name VARCHAR -- Name of the race
    );
    CREATE TABLE IF NOT EXISTS results (
        driverid VARCHAR, -- Unique identifier for the driver
        raceid VARCHAR -- Unique identifier for the race
    );
    CREATE TABLE IF NOT EXISTS drivers (
        forename VARCHAR, -- First name of the driver
        driverid VARCHAR -- Unique identifier for the driver
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS dorm (
        dorm_name VARCHAR, -- Name of the dormitory
        dormid VARCHAR -- Unique identifier for the dormitory
    );
    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid VARCHAR, -- Unique identifier for the dormitory
        amenid VARCHAR -- Unique identifier for the amenity available in the dormitory
    );
    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid VARCHAR, -- Unique identifier for the amenity
        amenity_name VARCHAR -- Name of the amenity available in the dormitory
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Genre (
        GenreID VARCHAR, -- Unique identifier for the genre
        Name VARCHAR -- Name of the genre
    );
    CREATE TABLE IF NOT EXISTS Track (
        AlbumId VARCHAR, -- Unique identifier for the album
        GenreID VARCHAR -- Unique identifier for the genre of the track
    );
    CREATE TABLE IF NOT EXISTS Album (
        Title VARCHAR, -- Title of the album
        AlbumId VARCHAR -- Unique identifier for the album
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS player (
        name_first VARCHAR, -- First name of the player
        name_last VARCHAR, -- Last name of the player
        player_id VARCHAR -- Unique identifier for the player
    );
    CREATE TABLE IF NOT EXISTS team (
        team_id_br VARCHAR, -- Unique identifier for the team
        name VARCHAR -- Name of the team
    );
    CREATE TABLE IF NOT EXISTS salary (
        player_id VARCHAR, -- Unique identifier for the player
        team_id VARCHAR, -- Unique identifier for the team
        year VARCHAR -- Year of the salary
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS employee (
        emp_fname VARCHAR, -- First name of the employee
        emp_num VARCHAR -- Unique identifier for the employee
    );
    CREATE TABLE IF NOT EXISTS department (
        dept_name VARCHAR, -- Name of the department
        dept_code VARCHAR -- Code for the department
    );
    CREATE TABLE IF NOT EXISTS course (
        crs_description VARCHAR, -- Description of the course
        crs_code VARCHAR -- Code for the course
    );
    CREATE TABLE IF NOT EXISTS CLASS (
        prof_num VARCHAR, -- Unique identifier for the professor
        crs_code VARCHAR -- Code for the course
    );
    CREATE TABLE IF NOT EXISTS professor (
        prof_office VARCHAR, -- Office of the professor
        emp_num VARCHAR, -- Unique identifier for the employee
        dept_code VARCHAR -- Code for the department
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR, -- Unique identifier for the product
        product_details VARCHAR -- Details of the product
    );
    CREATE TABLE IF NOT EXISTS order_items (
        order_id VARCHAR, -- Unique identifier for the order
        product_id VARCHAR -- Unique identifier for the product in the order
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR, -- Name of the customer
        customer_id VARCHAR -- Unique identifier for the customer
    );
    CREATE TABLE IF NOT EXISTS customer_orders (
        customer_id VARCHAR, -- Unique identifier for the customer
        order_id VARCHAR -- Unique identifier for the order
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS student (
        fname VARCHAR, -- First name of the student
        stuid VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS has_pet (
        stuid VARCHAR, -- Unique identifier for the student
        petid VARCHAR -- Unique identifier for the pet
    );
    CREATE TABLE IF NOT EXISTS pets (
        petid VARCHAR, -- Unique identifier for the pet
        pettype VARCHAR -- Type of the pet
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS station (
        name VARCHAR, -- Name of the station
        station_id VARCHAR -- Unique identifier for the station
    );
    CREATE TABLE IF NOT EXISTS train (
        train_id VARCHAR, -- Unique identifier for the train
        Name VARCHAR -- Name of the train
    );
    CREATE TABLE IF NOT EXISTS train_station (
        station_id VARCHAR, -- Unique identifier for the station
        train_id VARCHAR -- Unique identifier for the train
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS furniture_manufacte (
        Furniture_ID VARCHAR, -- Unique identifier for the furniture
        manufacturer_id VARCHAR -- Unique identifier for the manufacturer
    );
    CREATE TABLE IF NOT EXISTS manufacturer (
        name VARCHAR, -- Name of the manufacturer
        manufacturer_id VARCHAR -- Unique identifier for the manufacturer
    );
    CREATE TABLE IF NOT EXISTS furniture (
        Furniture_ID VARCHAR, -- Unique identifier for the furniture
        num_of_component INTEGER -- Number of components in the furniture
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS CLASS (
        class_code VARCHAR, -- Code for the class
        crs_code VARCHAR -- Code for the course
    );
    CREATE TABLE IF NOT EXISTS enroll (
        stu_num VARCHAR, -- Unique identifier for the student
        class_code VARCHAR -- Code for the class
    );
    CREATE TABLE IF NOT EXISTS department (
        dept_code VARCHAR, -- Code for the department
        dept_name VARCHAR -- Name of the department
    );
    CREATE TABLE IF NOT EXISTS course (
        crs_code VARCHAR, -- Code for the course
        dept_code VARCHAR -- Code for the department
    );
    CREATE TABLE IF NOT EXISTS student (
        stu_fname VARCHAR, -- First name of the student
        stu_num VARCHAR -- Unique identifier for the student
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Employee (
        name VARCHAR, -- Name of the employee
        eid VARCHAR -- Unique identifier for the employee
    );
    CREATE TABLE IF NOT EXISTS Certificate (
        eid VARCHAR, -- Unique identifier for the employee
        aid VARCHAR -- Unique identifier for the certificate
    );
    CREATE TABLE IF NOT EXISTS Aircraft (
        aid VARCHAR, -- Unique identifier for the aircraft
        name VARCHAR -- Name of the aircraft
    );
    '''
]

print(len(gpt_context_lvl1), len(gpt_context_lvl2), len(gpt_context_lvl3), len(gpt_context_lvl4))


0 0 30 30


# Generacion de ejemplos

In [ ]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")
from train.mistral7bbase.functions.data_functions import read_data, write_data, get_column_letter

df = read_data("examples")

template = """Your task is to generate 3 similar SQL examples with their respective question following the next information.
Database schema:
{context}

Principal question:
{question}

SQL Query:
{answer}


Your answer must be a list of JSON items having this keys:
"""
id = 287

In [ ]:
import pyperclip

current_context = df["context"][id]
current_question = df["question"][id]
current_answer = df["answer"][id]
t = template.format(context = current_context, question = current_question, answer = current_answer)
t += """[
	...,
	{
		question: The question generated starting with: The human is asking/requesting ...,
		answer: The correct SQL Query for the question generated  
	},
	...
]

Remember you have to make new examples, DO NOT repeat the same PRINCIPAL EXAMPLE.
Begin!
"""
pyperclip.copy(t)
print(f"ID: {id} ====", current_question)
id = id + 1

# Actualizando el dataset

#### Paso 1: Agregando las columnas nuevas con datos generados al dataset como:
- "relations" para las relaciones entre tablas
- "context" para las tablas con comentarios

In [463]:
# mis columnas:
# "RELATIONS"        | "CONTEXT"
# gpt_relations_lvl1 | gpt_context_lvl1
# gpt_relations_lvl2 | gpt_context_lvl2
# gpt_relations_lvl3 | gpt_context_lvl3
# gpt_relations_lvl4 | gpt_context_lvl4

# Mis dataframes:
# lvl1_df
# lvl2_df
# lvl3_df
# lvl4_df

lvl1_df["relations"] = gpt_relations_lvl1
lvl2_df["relations"] = gpt_relations_lvl2
lvl3_df["relations"] = gpt_relations_lvl3
lvl4_df["relations"] = gpt_relations_lvl4

lvl1_df["context"] = gpt_context_lvl1
lvl2_df["context"] = gpt_context_lvl2
lvl3_df["context"] = gpt_context_lvl3
lvl4_df["context"] = gpt_context_lvl4


#### Paso 2: Gabrando el dataset actualizado en un archivo de excel

In [ ]:
# necesito: 
# A instruction = question 
# B response = answer 
# C relations 
# D context

from  train.mistral7bbase.functions.data_functions import write_data, get_column_letter

i = 0
for _, row in lvl1_df.iterrows():
    write_data("train_2", (get_column_letter("train_2", "question"), i, row["question"]))
    write_data("train_2", (get_column_letter("train_2", "answer"), i, row["answer"]))
    write_data("train_2", (get_column_letter("train_2", "relations"), i, row["relations"]))
    write_data("train_2", (get_column_letter("train_2", "context"), i, row["context"]))
    i += 1
i += 1
for _, row in lvl2_df.iterrows():
    write_data("train_2", (get_column_letter("train_2", "question"), i, row["question"]))
    write_data("train_2", (get_column_letter("train_2", "answer"), i, row["answer"]))
    write_data("train_2", (get_column_letter("train_2", "relations"), i, row["relations"]))
    write_data("train_2", (get_column_letter("train_2", "context"), i, row["context"]))
    i += 1
i += 1
for _, row in lvl3_df.iterrows():
    write_data("train_2", (get_column_letter("train_2", "question"), i, row["question"]))
    write_data("train_2", (get_column_letter("train_2", "answer"), i, row["answer"]))
    write_data("train_2", (get_column_letter("train_2", "relations"), i, row["relations"]))
    write_data("train_2", (get_column_letter("train_2", "context"), i, row["context"]))
    i += 1
i += 1
for _, row in lvl4_df.iterrows():
    write_data("train_2", (get_column_letter("train_2", "question"), i, row["question"]))
    write_data("train_2", (get_column_letter("train_2", "answer"), i, row["answer"]))
    write_data("train_2", (get_column_letter("train_2", "relations"), i, row["relations"]))
    write_data("train_2", (get_column_letter("train_2", "context"), i, row["context"]))
    i += 1

#### Paso 3: Agregando el prompt para entrenamiento del modelo

  Leemos la la hoja de excel y generamos el chat template

In [34]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")
from train.mistral7bbase.functions.data_functions import read_data, write_data, get_column_letter, get_examples_text
from train.mistral7bbase.notebooks.chat_templates import get_chat_template

def write_texts_column(state, model):
	try:
		if state not in ["train", "test"]:
			raise ValueError(f"Sheet: {state} no se encuentra. Debe ser 'train' o 'test'")
		if model not in ["llama38b", "mistral7b"]:
			raise ValueError(f"Model {model} no válido. Debe ser 'llama38b' o 'mistral7b'")
				
		data_df = read_data(state)
		train_template, test_template = get_chat_template(model)
				
		if model == "mistral7b":
			letter = get_column_letter(state, "Mistral-7B-Instruct-v0.2-texts")
		elif model == "llama38b":
			letter = get_column_letter(state, "Meta-Llama-3-8B-Instruct-texts")
				
		for index, row in data_df.iterrows():
			context = row["context"]
			question = row["question"]
			answer = row["answer"]
			try:
				examples_text = get_examples_text(row["examples"])
			except:
				raise ValueError(f"Error en el dato con id === {index}")

			relations = ""
			if type(row["relations"]) == str:
				relations = row["relations"]
				
			if state == "train":
				texts_template = train_template
				data = texts_template.format(context=context, relations=relations, examples = examples_text, question=question, answer=answer)
			else:
				texts_template = test_template
				data = texts_template.format(context=context, relations=relations, examples = examples_text, question=question)
			write_data(state, (letter, index, data))
	except Exception as e:
        # Manejamos cualquier excepción que ocurra durante el proceso
		print(f"Ocurrió un error: {e}")
			

In [ ]:
# write_texts_column("train", "mistral7b")
write_texts_column("test", "llama38b")

In [2]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")
from train.mistral7bbase.functions.data_functions import read_data, write_data, get_column_letter

def write_difficulty_column(state):
    try:
        if state not in ["train", "test"]:
            raise ValueError(f"Sheet: {state} no se encuentra. Debe ser 'train' o 'test'")
        
        data_df = read_data(state)
        letter = get_column_letter(state, "difficulty")
        for index, row in data_df.iterrows():
            joins_number = str(row["answer"]).count("JOIN")
            if joins_number <= 1:
                difficulty = "easy"
            elif joins_number == 2:
                difficulty = "medium"
            else:
                difficulty = "hard"
            
            write_data(state, (letter, index, difficulty))
            
    except Exception as e:
        print("An error occurred:", e)

In [ ]:
write_difficulty_column("test")

In [52]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")
from train.mistral7bbase.functions.data_functions import read_data, write_data, get_column_letter
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

def write_tokens_column(state, model):
    try:
        if state not in ["train", "test"]:
            raise ValueError(f"Sheet: {state} no se encuentra. Debe ser 'train' o 'test'")
        
        
        data_df = read_data(state)
        tokens_column = get_column_letter(state, "tokens")
        
        if model == "mistral7b":
            txt_column = "Mistral-7B-Instruct-v0.2-texts"
        elif model == "llama38b":
            txt_column = "Meta-Llama-3-8B-Instruct-texts"
				
        for index, row in data_df.iterrows():
            tokens_num = tokenizer(row[txt_column], return_tensors="pt")["input_ids"].shape[1]
            print(tokens_num)
            write_data(state, (tokens_column, index, tokens_num))
            
    except Exception as e:
        print("An error occurred:", e)

c:\Users\lauth\OneDrive\Desktop\azure-vm\Repo\azure-vm\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# write_texts_column("train", "mistral7b")
write_tokens_column("train", "llama38b")